# Naive Bayes Classifier for Natural Language Processing

This article will review a bread-and-butter natural language processing (NLP) model known as the Naive Bayes Classifier. We will analyse a series of tweets with binary outcomes from the [kaggle disaster dataset](https://www.kaggle.com/c/nlp-getting-started). These tweets show peoples' online reaction to natural disasters world wide. The goal is to build a model to automatically classify which tweets pertain to disasters and which do not.

In this post we will learn

1): What is a Naive Bayes classifier?

2): Python scikit-learn implementation

3): How do we measure model performance?

##  What is a Naive Bayes Classifier?

Training a model to label a sample of text as positive / negative
is known as Natural Language Processing (NLP) binary classification.

A common choice of model for such a task is the Naive Bayes classifier.
Using [Bayes theorem](https://medium.com/@theflyingmantis/text-classification-in-nlp-naive-bayes-a606bf419f8c
),
we can formulate our outcome variable as the probability of a certain class 0 or 1 given some input text X. In our case, a tweet referring to a recent natural disaster is labeled 1 and a neutral tweet 0. In maths speak this is known as the Naive Bayes probability P(C_j | X), which is calculated from Bayes theorem as

P(C_j | X) = P(X | C_j) P(C_j) / P(X).

From the above, we have three quantities to calculate.

#### __P(X)__: 

Give that all probabilities have P(X) as the denominator, this constant can be disregarded.

#### __P(C_j):__ 

This is simple enough. It is just the relative
fraction of class _i_ in the data set (i.e.
for the positive class, this is the number of
positives divided by the total number of samples
in the training data).

#### __P(X | C_j):__

Representing the input as a set of features
x1, x2 ...xn, P(X) = P(x1, x2...xn).
We can rewrite P(X | C_j) as P(x1,x2...xn | C_j).
Now comes the trick. The word _Naive_ in Naive Bayes
classifiers means that we make the assumption that
probabilities of all words are independent of
each other. This chiefly means that we assume
the order of the words doesn't matter (an incorrect assumption
but one that often doesn't introduce too much error), but
 also allows us to rewrite our conditional
 probability as

P(x1,x2...xn | C_j) = P(x1|C_j) P(x2 | C_j) .... P(xn | C_j).

The probability of word 1 given class j, P(x1 | C_j), is then

P(xi | C_j) = count (xi, C_j) / sum_k (xk, C_j).

i.e. this counts all occurrences of word xi in
all inputs of class C_j and divides them by the
sum of counts of all words in the vocabulary.
Laplace smoothing can be used to mitigate the
effects of zero occurrences of words and
dividing by zero.

In summary, the Naive Bayes probability for
input X, P(X | C_j) P(C_j), is calculated by
multiplying together P(xi | C_j) for all words
in our vocabulary and multiplying the result by
P(C_J). Lets now set one up using sci-kit learn.

## Python Scikit-learn Implementation:

Python's scitkit-learn module includes excellent Naive Bayes functionality for building and NLP classifier. To begin, lets load our input data.

In [ ]:
#import dependencies
#for data ingestion manipulation
import pandas as pd

#to split data into train/test samples
from sklearn.model_selection import train_test_split

#to generate our sparse matrix of word occurrence counts
from sklearn.feature_extraction.text import CountVectorizer

#to normalise over tweet length and word overuse
from sklearn.feature_extraction.text import TfidfTransformer

#the Naive Bayes classifier
from sklearn.naive_bayes import MultinomialNB

#combine word count, transformation, fit / predict steps into a single pipeline
from sklearn.pipeline import Pipeline

# to compute the ROC curve and Area AUC
from sklearn import metrics

# to visualise the ROC
import matplotlib.pylab as plt

In [ ]:
#load data
df = pd.read_csv('./nlp_data/train.csv')
X, y = list(df['text']), list(df['target'])

In [ ]:
#split into train test samples
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

### Get the word counts for the train data

We now must construct the probabilities _P(X|C_j)_ for each word / class combination. We therefore need the occurrence counts of each word in each of our input samples.

In [ ]:
#Get the word counts for the train data
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)
X_train_counts.shape

### Word count transformations

Occurrence count is a good start but there is an issue: longer tweets will have higher 
average count values than shorter tweets, even though they might talk about the same topics.

To avoid these potential discrepancies it suffices to divide the number of 
occurrences of each word in a tweet by the total number of words in 
the tweet: these new features are called tf for Term Frequencies.

Another refinement on top of tf is to downscale weights for words that occur 
in many tweets in the sample and are therefore less informative than those 
that occur only in a smaller portion of the tweets (e.g words like "the", "and" etc)

In [ ]:
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

### Fit the naive bayes classifier

In [ ]:
clf = MultinomialNB().fit(X_train_tfidf, y_train)

### predict the labels for the test data

First we get the word counts and apply the transformations as to the test data

In [ ]:
X_test_counts = count_vect.transform(X_test)
X_test_tfidf = tfidf_transformer.transform(X_test_counts)
y_pred = clf.predict(X_test_tfidf)

### Build a pipeline

The count, transformation and predict steps can be combined into one using scikit-learns pipeline functionality.

In [ ]:
text_clf = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', MultinomialNB()),])
text_clf.fit(X_train, y_train)
y_pred = text_clf.predict(X_test)

## How do we Measure Model Performance?

To measure the performance of our classifier, we could simply use an accuracy metric from our true positive (TP), false positive (FP), true negative (TN) and false negative (FN) counts. This is defined as

`accuracy = TP + FP / (TP + FP + TN + FN)`

This is a common performance metric for a classifier but is suceptible to class imballances. When testing for a disease for example, most of the training sample will likely not be infected. In these cases a classifier that always predicted negative would achieve a high accuracy score despite being useless. It is better therefore to use a metric that combines the following concepts:

#### Precision = TP / (TP + FP)
Is the fraction of positive classifications that were correct.

#### Recall = TP / (TP + FN) 
Is the fraction of positive that were identified by the classifier.


Remember that classifiers output the probability of an input belonging to each class. We then choose an arbitrary threshold probability to say whether the probability indicates one class or another. A good way of measuring the performance of a classifier is to construct the [ROC](https://en.wikipedia.org/wiki/Receiver_operating_characteristic) curve trailing various threshold probabilities, and measuring the area under the curve (AUC). Fortunately, scikit-learn comes to the rescue again.

In [ ]:
fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred, pos_label=1)
auc = metrics.auc(fpr, tpr)
print('ROC curve AUC = '+str(auc))

### Visualising ROC curve

We can plot the roc curve as true positive rate vs false positive rate using matplotlib.

In [ ]:
import matplotlib.pylab as plt
fig = plt.figure()
ax1 = fig.add_subplot(111)
idx = np.argsort(fpr)
ax1.plot(fpr[idx],tpr[idx],label='Naive Bayes Classifier')
ax1.set_xlabel('False Positive Rate')
ax1.set_ylabel('True Positive Rate')
plt.show()

# Summary

Here we have quickly reviewed the stats behind the Naive Bayes text classification algorithm, constructed a Naive Bayes classifier in Python and written code to measure its performance.

Naive Bayes models are one of the simpler text classification models and often outperformed by neural net type methods (long short term memory networks are a good choice for text classification). Despite this, simpler models mean less potential for the model to overfit, a smaller training sample, and the simpler model interpret-ability. Opting for a simpler classifier model over a neural net can save lots of pain further down the line and will often get you then answer you need, and perform almost as well.